![book](https://raw.githubusercontent.com/ageron/tensorflow-safari-course/master/images/intro_to_tf_course.png)

**Try not to peek at the solutions when you go through the exercises. ;-)**

First let's make sure this notebook works well in both Python 2 and Python 3:

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow basics

In [2]:
import tensorflow as tf
tf.__version__

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.7.0'

## Construction Phase

In [3]:
>>> a = tf.constant(3)
>>> b = tf.constant(5)
>>> s = a + b

In [4]:
a

<tf.Tensor 'Const:0' shape=() dtype=int32>

In [5]:
b

<tf.Tensor 'Const_1:0' shape=() dtype=int32>

In [6]:
s

<tf.Tensor 'add:0' shape=() dtype=int32>

In [7]:
tf.get_default_graph()

In [9]:
>>> graph = tf.Graph()
>>> with graph.as_default():
...     a = tf.constant(3)
...     b = tf.constant(5)
...     s = a + b
...

## Execution Phase

In [10]:
>>> with tf.Session(graph=graph) as sess:
...     result = s.eval()
...
>>> result

8

In [11]:
>>> with tf.Session(graph=graph) as sess:
...     result = sess.run(s)
...
>>> result

8

In [14]:
>>> with tf.Session(graph=graph) as sess:
...     result = sess.run([a,b,s])
...
>>> result

[3, 5, 8]

## Exercise 1

![Exercise](https://c1.staticflickr.com/9/8101/8553474140_c50cf08708_b.jpg)

1.1) Create a simple graph that calculates $ c = \exp(\sqrt 8 + 3) $.

**Tip**: TensorFlow's API documentation is available at:
https://www.tensorflow.org/versions/master/api_docs/python/

In [37]:
a = tf.constant(8, 'float32')
a

<tf.Tensor 'Const_8:0' shape=() dtype=float32>

In [23]:
b = tf.constant(3, 'float32')

In [24]:
c = tf.exp(tf.sqrt(a)+b)

1.2) Now create a `Session()` and evaluate the operation that gives you the result of the equation above:

In [27]:
Run = tf.Session()
Run.run(c)

339.82382

In [32]:
with tf.Session() as sess:
    result = sess.run(c)
print(result)

339.82382


In [34]:
with tf.Session() as sess:
    result = sess.run([a, b, c])
print(result)

[8.0, 3.0, 339.82382]


1.3) Create a graph that evaluates and prints both $ b = \sqrt 8 $ and $ c = \exp(\sqrt 8 + 3) $. Try to implement this in a way that only evaluates $ \sqrt 8 $ once.

In [58]:
graph = tf.Graph() 
with graph.as_default():
    a = tf.constant(8, 'float32')
    b = tf.constant(3, 'float32')
    c = tf.sqrt(a)
    d = tf.exp(c + b)

In [59]:
with tf.Session(graph=graph) as sess:
    result = sess.run([a, b, c, d])
print(result)

[8.0, 3.0, 2.828427, 339.82382]


In [61]:
with tf.Session(graph=graph):
    d_evl = d.eval()
print(d_evl)

339.82382


1.4) The following code is needed to display TensorFlow graphs in Jupyter. Just run this cell then visualize your graph by calling `show_graph(`_your graph_`)`:

In [44]:
import numpy as np
from IPython.display import display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def=None, max_const_size=32):
    """Visualize TensorFlow graph."""
    graph_def = graph_def or tf.get_default_graph()
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [45]:
show_graph(graph)

Try not to peek at the solution below before you have done the exercise! :)

![thinking](https://upload.wikimedia.org/wikipedia/commons/0/06/Filos_segundo_logo_%28flipped%29.jpg)

## Exercise 1 - Solution

1.1)

In [47]:
graph = tf.Graph()
with graph.as_default():
    c = tf.exp(tf.add(tf.sqrt(tf.constant(8.)), tf.constant(3.)))
    # or simply...
    c = tf.exp(tf.sqrt(8.) + 3.)

1.2)

In [48]:
with tf.Session(graph=graph):
    c_val = c.eval()

In [49]:
c_val

339.82382

1.3)

In [50]:
graph = tf.Graph()
with graph.as_default():
    b = tf.sqrt(8.)
    c = tf.exp(b + 3)

In [51]:
with tf.Session(graph=graph) as sess:
    b_val, c_val = sess.run([b, c])

In [52]:
b_val

2.828427

In [53]:
c_val

339.82382

**Important**: the following implementation gives the right result, but it runs the graph twice, once to evaluate `b`, and once to evaluate `c`.  Since `c` depends on `b`, it means that `b` will be evaluated twice. Not what we wanted.

In [54]:
# WRONG!
with tf.Session(graph=graph):
    b_val = b.eval()  # evaluates b
    c_val = c.eval()  # evaluates c, which means evaluating b again!

In [55]:
b_val

2.828427

In [56]:
c_val

339.82382

1.4)

In [ ]:
show_graph(graph)